In [4]:
from mysql.connector import connect
from elasticsearch import Elasticsearch, helpers
from ssl import create_default_context
import json
import pandas as pd
import datetime

from ssl import create_default_context, CERT_NONE
from elasticsearch import Elasticsearch

import warnings
warnings.filterwarnings('ignore')

from IPython.display import display, HTML 
display(HTML("<style>.container { width:100% !important; }</style>"))

import os
from elasticsearch import Elasticsearch, helpers
from ssl import create_default_context, CERT_NONE
import json
import pandas as pd
import datetime
import logging
import re

import sys

sys.path.append(r'D:\markding_git\big-data-ai-integration-platform\airflow\dags\code')
import es_mapping
from connection import ElasticSearchConnectionManager


# Alphabet generator

In [2]:
# use the string module
import string
alphabet_list = list(string.ascii_lowercase)

print(alphabet_list)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


# ES connection settup

In [5]:
# def create_es_connection():
#     # Load certificate path and credentials from environment variables
#     cafile_path = os.getenv('ES_CAFILE_PATH', r"D:\markding_git\big-data-ai-integration-platform\search_system\es_certs\http_ca.crt")
#     es_host = os.getenv('ES_HOST', 'http://localhost:9200')
#     es_user = os.getenv('ES_USER', 'elastic')
#     es_password = os.getenv('ES_PASSWORD', 'gAcstb8v-lFCVzCBC__a')
#     # uWzW*VOzBZknR-Jt3T7i
#     context = create_default_context(cafile=cafile_path)
#     # context.check_hostname = True  # Ensure hostname is checked
#     # context.verify_mode = CERT_REQUIRED  # Ensure the certificate is verified

#     context.check_hostname = False
#     context.verify_mode = CERT_NONE
    
#     es = Elasticsearch(
#                 [es_host],
#                 http_auth=(es_user, es_password),
#                 verify_certs=False,
#                 )
#     return es

# es = create_es_connection()
es = ElasticSearchConnectionManager.get_instance()

print(es.info())

es_health = es.health_report()
print(es_health['status'])
indicators = es_health['indicators']
[f"status_of_[{i}] : {indicators[i]['status']}" for i in indicators.keys()]

2025-09-10 12:37:10,073 [INFO] connection - Attempting to connect to Elasticsearch at http://localhost:9200 with CA file  (Attempt 1)
2025-09-10 12:37:10,368 [INFO] connection - Elasticsearch connection established to http://localhost:9200.


{'name': '44b52d9c8e08', 'cluster_name': 'docker-cluster', 'cluster_uuid': '0XOBCloyQGW6WDdj1nbg3Q', 'version': {'number': '8.12.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '48a287ab9497e852de30327444b0809e55d46466', 'build_date': '2024-02-19T10:04:32.774273190Z', 'build_snapshot': False, 'lucene_version': '9.9.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}
yellow


['status_of_[master_is_stable] : green',
 'status_of_[repository_integrity] : green',
 'status_of_[disk] : green',
 'status_of_[shards_capacity] : green',
 'status_of_[shards_availability] : yellow',
 'status_of_[slm] : green',
 'status_of_[ilm] : green']

In [5]:
def es_mapping(df, column_types_dict):
    """
    Generate Elasticsearch mapping based on the DataFrame and a column-to-Python type dictionary.

    Args:
        df (pandas.DataFrame): The DataFrame to be indexed.
        column_types_dict (dict): Dictionary mapping columns to Python types.

    Returns:
        dict: Elasticsearch mapping dictionary.
    """

    # Define a mapping from Python types to Elasticsearch types
    type_conversion = {
        'object': 'keyword',  # Typically strings in Pandas
        'int64': 'integer',
        'int32': 'integer',
        'float64': 'float',
        'float32': 'float',
        'bool': 'boolean',
        'datetime64[ns]': 'date',
        'timedelta[ns]': 'long',
        'category': 'keyword',
    }

    # Generate the Elasticsearch mapping using the provided column types
    mapping = {
        "mappings": {
            "properties": {
                col: {
                    "type": type_conversion.get(column_types_dict[col], "text"),  # Default to "text"
                    "ignore_malformed": True if type_conversion.get(column_types_dict[col]) in ["integer", "float"] else False
                }
                for col in df.columns
            }
        }
    }

    logger.info(f"Mapping: {mapping}")
    return mapping

In [6]:
# Function to create a MySQL connection
def mysql_connection_whole_corp():
    return connect(host='localhost',
                   port=3307,
                   user='root',
                   password='!QAZ2wsx',
                   database='whole_corp')

con = mysql_connection_whole_corp()
cursor = con.cursor()

cursor.execute('select * from whole_corp limit 100')
result = cursor.fetchall()

df = pd.DataFrame(result, columns= cursor.column_names)


In [16]:
es.indices.get_alias(index="*")


ObjectApiResponse({'wholecorp_clusters_vector': {'aliases': {}}, 'whole_corp': {'aliases': {}}})

In [18]:
es.search(index='whole_corp', body={"query": {"match_all": {}}})

ObjectApiResponse({'took': 5, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 10000, 'relation': 'gte'}, 'max_score': 1.0, 'hits': [{'_index': 'whole_corp', '_id': 'lSBbJJkBE3HYBffYCj14', '_score': 1.0, '_source': {'統一編號': '47737059', '公司名稱': '向日葵早午餐', '負責人': '范鳳仙', '登記地址': '屏東縣來義鄉古樓村新隆路204號1樓', '資本額': '3000.0', '營業項目及代碼表': '餐館業', '縣市名稱': '屏東縣', '區域名稱': '來義鄉', '縣市區域': '屏東縣來義鄉', '類別_全': '餐館業', '官網': '\\N', '電話': '\\N\r'}}, {'_index': 'whole_corp', '_id': 'lyBbJJkBE3HYBffYCj14', '_score': 1.0, '_source': {'統一編號': '47737797', '公司名稱': '瘋厚牛排餐館', '負責人': '黃俊義', '登記地址': '屏東縣東港鎮明德一路２９６號1樓', '資本額': '100000.0', '營業項目及代碼表': '飲料店業,餐館業,其他餐飲業', '縣市名稱': '屏東縣', '區域名稱': '東港鎮', '縣市區域': '屏東縣東港鎮', '類別_全': '飲料店業,餐館業,其他餐飲業', '官網': '\\N', '電話': '\\N\r'}}, {'_index': 'whole_corp', '_id': 'mCBbJJkBE3HYBffYCj14', '_score': 1.0, '_source': {'統一編號': '47739079', '公司名稱': '薩姆拉灣外燴', '負責人': '楊信義', '登記地址': '屏東縣潮州鎮三和里福樂街41號', '資本額': '3000.0', '營業項目及代碼表'

# Mapping for each tables

# Create index with mapping in ES

In [19]:
if __name__ == '__main__':
    es = create_es_connection()
    index_name = 'whole_corp'
    if es.indices.exists(index = index_name):
        if str(input('Index {index_name} already exists. Delete? [y/n]').lower()) == 'y':
            es.indices.delete(index=index_name)
        else:
            print('count:', es.count(index = index_name))

    response = es.indices.create(index=index_name, body=corp_mapping())
    
    # Checking mapping in each index
    print(f"/n=============response, es.indices.get_mapping(index={index_name})=============")
    print(response, es.indices.get_mapping(index=index_name))
    

/n=============response, es.indices.get_mapping(index=whole_corp)=============
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'whole_corp'} {'whole_corp': {'mappings': {'properties': {'公司名稱': {'type': 'text', 'fields': {'raw': {'type': 'keyword'}}, 'analyzer': 'traditional_chinese_analyzer'}, '區域名稱': {'type': 'keyword'}, '官網': {'type': 'keyword', 'index': False}, '營業項目及代碼表': {'type': 'text', 'fields': {'raw': {'type': 'keyword'}}, 'analyzer': 'traditional_chinese_analyzer'}, '登記地址': {'type': 'text'}, '統一編號': {'type': 'keyword'}, '縣市區域': {'type': 'keyword'}, '縣市名稱': {'type': 'keyword'}, '負責人': {'type': 'text', 'fields': {'keyword': {'type': 'keyword'}}}, '資本額': {'type': 'double'}, '電話': {'type': 'keyword'}, '類別_全': {'type': 'text', 'fields': {'raw': {'type': 'keyword'}}, 'analyzer': 'traditional_chinese_analyzer'}}}}}


# Adjusted mapping

In [19]:
'''-----------------------------Create mapping for user----------------------------------'''
def corp_mapping():
    # Define the settings for the custom analyzer
    body = {
        "properties": {
          "cluster": {
            "type": "keyword"
          },
          "vector": {
            "type": "dense_vector",
            "dims": 100    # // must match your model output
          }
        }
      }

    return body


In [20]:
es.indices.put_mapping(index='whole_corp', body=corp_mapping())

ObjectApiResponse({'acknowledged': True})

# Get index mapping

In [9]:
es.indices.get_mapping(index='whole_corp')

ObjectApiResponse({'whole_corp': {'mappings': {'properties': {'cluster': {'type': 'keyword'}, 'vector': {'type': 'dense_vector', 'dims': 100, 'index': True, 'similarity': 'cosine'}, '公司名稱': {'type': 'text', 'fields': {'raw': {'type': 'keyword'}}, 'analyzer': 'traditional_chinese_analyzer'}, '區域名稱': {'type': 'keyword'}, '官網': {'type': 'keyword', 'index': False}, '營業項目及代碼表': {'type': 'text', 'fields': {'raw': {'type': 'keyword'}}, 'analyzer': 'traditional_chinese_analyzer'}, '登記地址': {'type': 'text'}, '統一編號': {'type': 'keyword'}, '縣市區域': {'type': 'keyword'}, '縣市名稱': {'type': 'keyword'}, '負責人': {'type': 'text', 'fields': {'keyword': {'type': 'keyword'}}}, '資本額': {'type': 'double'}, '電話': {'type': 'keyword'}, '類別_全': {'type': 'text', 'fields': {'raw': {'type': 'keyword'}}, 'analyzer': 'traditional_chinese_analyzer'}}}}})

In [9]:
body = {
    "query":{
        "bool":{
        "must":{
            'exists': { "field": "vector"}
        }
    }
    }
}

In [10]:
es.search(index='whole_corp', body=body, size=1)

ObjectApiResponse({'took': 197, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}})

# Import data

In [ ]:
es = create_es_connection()
for index_name in ['whole_corp']:
    try:
        print(f'index name : [{index_name}]',es.count(index=index_name.lower()))
    except Exception as e:
        print(f'\n==============no values in index name [{index_name}]================')
es.close()

index name : [whole_corp] {'count': 1144129, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}


In [ ]:
es.indices.delete(index='whole_corp')

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [whole_corp/ThaN78eZT9KDRpBMFHo8fw] already exists')

# Data import to ES from mysql

In [ ]:
'''=============================================['userESG', 'lifeCircleESG', 'courseESG']==============================================='''
# Function to create a MySQL connection
def mysql_connection_whole_corp():
    return connect(host='localhost',
                   port=3307,
                   user='root',
                   password='!QAZ2wsx',
                   database='whole_corp')


def json_serial(obj):
    """JSON serializer for objects not serializable by default json code"""
    if isinstance(obj, datetime.datetime):
        return obj.isoformat()
    if isinstance(obj, bytearray):
        return obj.decode('utf-8')
    raise TypeError ("Type %s not serializable" % type(obj))
    

# Generator function to fetch data in batches from MySQL
def fetch_data(cursor, batch_size=1000):
    while True:
        rows = cursor.fetchmany(batch_size)
        if not rows:
            break
        # Convert each row to JSON serializable format
        cleaned_rows = []
        for row in rows:
            cleaned_row = {k: json_serial(v) if isinstance(v, (datetime.datetime, bytearray)) else 0.0 if (k == '資本額') and (v == '\\N') else v for k, v in row.items()}
            cleaned_rows.append(cleaned_row)
        yield cleaned_rows
    

def elastic_import(es, actions):
    response, errors  = helpers.bulk(es, actions, raise_on_error=False)
    if errors:
        print(f'==================================errors: ===============================\n{errors}')
        
    
# Main function to control the flow of the script
def main(table_name, database = None):
    # Connect to MySQL
    
    mysql_conn = mysql_connection_whole_corp() 
        
    cursor = mysql_conn.cursor(dictionary=True)
    
    # Define the SQL query and execute it
    sql_query = f"SELECT * FROM {table_name}"
    cursor.execute(sql_query)
    
    # Connect to Elasticsearch
    es = create_es_connection()

    # Define the Elasticsearch index where the data will be stored
    es_index = table_name.lower()
    
    # Fetch data from MySQL and prepare it for Elasticsearch
#     if es.indices.exists(index = table_name):
#         return 'the index already exists'
    
    for batch in fetch_data(cursor):
        actions = [
            {
                "_index": es_index,
                "_source": row,
            } for row in batch
        ]
        
        # Bulk index the data in Elasticsearch
        elastic_import(es, actions)

    # Cleanup
    cursor.close()
    mysql_conn.close()
    return actions
    
    
if __name__ == "__main__":
    
    #Define paramenter
    database = 'whole_corp'
    table_name = 'whole_corp'
    es_data = main(table_name,database)


==================================errors: ===============================
[{'index': {'_index': 'whole_corp', '_id': '1CmiJ5kBifl7WjgxL4wN', 'status': 400, 'error': {'type': 'document_parsing_exception', 'reason': "[1:169] failed to parse field [資本額] of type [double] in document with id '1CmiJ5kBifl7WjgxL4wN'. Preview of field's value: ' '", 'caused_by': {'type': 'number_format_exception', 'reason': 'empty String'}}}}]
==================================errors: ===============================
[{'index': {'_index': 'whole_corp', '_id': 'wCmiJ5kBifl7WjgxMJT-', 'status': 400, 'error': {'type': 'document_parsing_exception', 'reason': "[1:157] failed to parse field [資本額] of type [double] in document with id 'wCmiJ5kBifl7WjgxMJT-'. Preview of field's value: ' '", 'caused_by': {'type': 'number_format_exception', 'reason': 'empty String'}}}}, {'index': {'_index': 'whole_corp', '_id': 'TSmiJ5kBifl7WjgxMZU6', 'status': 400, 'error': {'type': 'document_parsing_exception', 'reason': "[1:175] failed

KeyboardInterrupt: 

# Update index with specific field

In [12]:
from elasticsearch import helpers
import numpy as np
import pandas as pd
import datetime, boto3
import es_mapping
from connection import ElasticSearchConnectionManager
import sys

sys.path.append(r'C:\Users\mark.ding\big-data-ai-integration-platform\common')
from logger import initlog

logger = initlog(__name__)

# JSON serializer for datetime and bytearray objects
def json_serial(obj):
    if isinstance(obj, datetime.datetime):
        return obj.isoformat()
    if isinstance(obj, bytearray):
        return obj.decode('utf-8')
    raise TypeError(f"Type {type(obj)} not serializable")

# Fetch data from MySQL in batches
def fetch_data(cursor, batch_size=1000):
    while True:
        rows = cursor.fetchmany(batch_size)
        if not rows:
            break
        cleaned_rows = [
            {k: json_serial(v) if isinstance(v, (datetime.datetime, bytearray)) else v for k, v in row.items()}
            for row in rows
        ]
        logger.debug(f"Fetched {len(cleaned_rows)} rows from MySQL.")
        yield cleaned_rows

# Search and update document in Elasticsearch
def search_and_update_field(es, index, source_id, vector_val, cluster_val):
    try:
        search_query = {"query": {"term": {"統一編號": source_id}}}
        search_response = es.search(index=index, body=search_query)
        hits = search_response['hits']['hits']
        
        if not hits:
            logger.info(f"Document with source ID {source_id} does not exist.")
            return False

        for hit in hits:
            doc_id = hit['_id']
            response = es.update(index=index, id=doc_id, body={"doc": {
                "vector": vector_val,
                "cluster" :cluster_val
            }})
            logger.debug(f"Document with source ID {source_id} updated: {response}")
            return True

    except Exception as e:
        logger.error(f"Error updating document with source ID {source_id}: {e}")
        return False

# Bulk import data to Elasticsearch
def elastic_import(es, actions):
    try:
        response, errors = helpers.bulk(es, actions, raise_on_error=False)
        logger.info(f"Imported {response} documents to Elasticsearch, errors: {errors}")
    except Exception as e:
        logger.error(f"Bulk import errors: {e}")

# Update data in Elasticsearch
def update_data_to_es(es, cursor, es_index, table_name):
    updated_count = 0
    create_data_count = 0
    actions = []

    for batch in fetch_data(cursor):
        logger.debug(f"Processing batch with {len(batch)} records.")
        
        df = pd.DataFrame(batch)
        df = df.where(pd.notnull(df), None)  # Replace NaN with None
        df = df.replace({np.nan: None})  # Replace NaN with None

        batch = df.to_dict(orient='records')

        for row in batch:
            source_id = row['統一編號']
            # logger.info(f"Updating document with source ID {source_id}...")
            try:
                print('row:========', row)
                doc_exists = search_and_update_field(es, es_index, source_id, row.get('vector'), row.get('cluster'))
                if not doc_exists:
                    actions.append({"_index": es_index, "_source": row})
                    create_data_count += 1
                    logger.debug(f"Document with source ID {source_id} created.")
                else:
                    updated_count += 1
                    logger.debug(f"Document with source ID {source_id} updated.")

            except Exception as e:
                logger.error(f"Error updating document with source ID {source_id}: {e}")

        if actions:
            logger.info(f"Importing {len(actions)} documents to Elasticsearch index: [{es_index}]...]")
            elastic_import(es, actions)
            actions = []  # Clear actions after bulk import

    logger.info(f"[{table_name}] total documents updated: {updated_count}, and created: {create_data_count}")
    return updated_count, create_data_count

# Main function to handle the ETL process
def etl_process(table_name, es, es_index):
    logger.info(f"Starting ETL process for table: {table_name}.")
    
    # Create MySQL connection
    mysql_conn = ElasticSearchConnectionManager.mysql_connection_whole_corp() 
    cursor = mysql_conn.cursor(dictionary=True)

    date_now_python = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    date_now_sql_query = "SELECT now()"
    cursor.execute(date_now_sql_query)
    date_now_sql = cursor.fetchone()
    date_now_sql = date_now_sql['now()'].strftime("%Y-%m-%d %H:%M:%S")

    # Check if date time is the same in Python and SQL
    if date_now_python != date_now_sql:
        logger.info("Date time is not the same: %s and %s", date_now_python, date_now_sql)
        cursor.execute("SET time_zone = 'Asia/Taipei';")
    else:
        logger.info("Date time is the same: %s", date_now_python)

    # Query to fetch updated records
    sql_query_updatedAt = f"SELECT * FROM {table_name}"
    cursor.execute(sql_query_updatedAt)

    # Update data in Elasticsearch
    update_data_count, create_data_count = update_data_to_es(es, cursor, es_index, table_name)

    mysql_conn.close()
    logger.info(f"ETL process for table: {table_name} completed with {update_data_count} updates and {create_data_count} creations.")
    return update_data_count, create_data_count


def create_index_if_not_exists(es, index_name):
    try:
        es.indices.get_mapping(index=index_name)
        logger.info(f"Index {index_name} already exists.")
    except Exception as e:
        logger.warning(f"Index {index_name} does not exist. Creating new index.")
        mapping = es_mapping.corp_mapping()
        if mapping:
            es.indices.create(index=index_name, body=mapping)
            logger.info(f"Index {index_name} created with the specified mapping.")
        else:
            logger.error(f"Failed to create index {index_name}. No valid mapping found.")


def main():
    logger.info("ETL process started.")
    es = ElasticSearchConnectionManager.get_instance()
    
    table_name = 'wholecorp_clusters_vector'
    
    index_name = table_name.lower()
    
    create_index_if_not_exists(es, index_name)
    
    update_data_count, create_data_count = etl_process(table_name, es, index_name)
    
    logger.info(f"Data update for {table_name} completed, with update_data_count: {update_data_count}, create_data_count: {create_data_count}")
    
    logger.info("ETL process finished.")


if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        logger.error(f"ETL process failed due to: {str(e)}", exc_info=True)


2025-09-10 10:32:31,713 [INFO] __main__ - ETL process started.


2025-09-10 10:32:31,848 [INFO] __main__ - Index wholecorp_clusters_vector already exists.
2025-09-10 10:32:31,848 [INFO] __main__ - Starting ETL process for table: wholecorp_clusters_vector.
2025-09-10 10:32:31,895 [INFO] __main__ - Date time is not the same: 2025-09-10 10:32:31 and 2025-09-10 02:32:31
2025-09-10 10:32:31,956 [INFO] __main__ - Document with source ID 88501417 does not exist.
2025-09-10 10:32:32,010 [INFO] __main__ - Document with source ID 53006067 does not exist.
2025-09-10 10:32:32,060 [INFO] __main__ - Document with source ID 70517508 does not exist.
2025-09-10 10:32:32,112 [INFO] __main__ - Document with source ID 97945095 does not exist.


row:======== {'統一編號': '88501417', 'cluster': 8, 'vector': '[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]'}
row:======== {'統一編號': '53006067', 'cluster': 8, 'vector': '[0.00019179812754173336, -7.3557461837601384e-06, 8.231371875547439e-05, 0.0006201981267839028, 0.0011565673919607523, 0.008598308163353212, -0.003907957851022458, 0.0013754783827217372, -0.000411765917983247, 0.0016324419318472809, 0.001775272722444102, 0.0017426998978320146, 0.001301028772783204, 0.0006673750901778963, 0.0020041126503924887, 0.001834178925622977, 0.001533191

2025-09-10 10:32:32,164 [INFO] __main__ - Document with source ID 16231061 does not exist.
2025-09-10 10:32:32,246 [INFO] __main__ - Document with source ID 89637010 does not exist.
2025-09-10 10:32:32,299 [INFO] __main__ - Document with source ID 86697344 does not exist.


row:======== {'統一編號': '89637010', 'cluster': 8, 'vector': '[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]'}
row:======== {'統一編號': '86697344', 'cluster': 8, 'vector': '[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

...

2025-09-10 10:33:11,704 [INFO] __main__ - Document with source ID 26243100 does not exist.
2025-09-10 10:33:11,751 [INFO] __main__ - Document with source ID 27567485 does not exist.
2025-09-10 10:33:11,800 [INFO] __main__ - Document with source ID 24241417 does not exist.
2025-09-10 10:33:11,847 [INFO] __main__ - Document with source ID 12876131 does not exist.


row:======== {'統一編號': '26243100', 'cluster': 8, 'vector': '[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]'}
row:======== {'統一編號': '27567485', 'cluster': 8, 'vector': '[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

2025-09-10 10:33:11,895 [INFO] __main__ - Document with source ID 77128504 does not exist.
2025-09-10 10:33:11,943 [INFO] __main__ - Document with source ID 99035733 does not exist.
2025-09-10 10:33:11,944 [INFO] __main__ - Importing 786 documents to Elasticsearch index: [wholecorp_clusters_vector]...]


row:======== {'統一編號': '77128504', 'cluster': 8, 'vector': '[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]'}
row:======== {'統一編號': '99035733', 'cluster': 8, 'vector': '[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

2025-09-10 10:33:12,335 [INFO] __main__ - Imported 0 documents to Elasticsearch, errors: [{'index': {'_index': 'wholecorp_clusters_vector', '_id': 'AOV4MZkBCETuqezfTiu5', 'status': 400, 'error': {'type': 'document_parsing_exception', 'reason': '[1:551] failed to parse: Failed to parse object: expecting token of type [VALUE_NUMBER] but found [END_OBJECT]', 'caused_by': {'type': 'parsing_exception', 'reason': 'Failed to parse object: expecting token of type [VALUE_NUMBER] but found [END_OBJECT]', 'line': 1, 'col': 551}}}}, {'index': {'_index': 'wholecorp_clusters_vector', '_id': 'AeV4MZkBCETuqezfTivB', 'status': 400, 'error': {'type': 'document_parsing_exception', 'reason': '[1:2342] failed to parse: Failed to parse object: expecting token of type [VALUE_NUMBER] but found [END_OBJECT]', 'caused_by': {'type': 'parsing_exception', 'reason': 'Failed to parse object: expecting token of type [VALUE_NUMBER] but found [END_OBJECT]', 'line': 1, 'col': 2342}}}}, {'index': {'_index': 'wholecorp_cl